In [1]:
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

# 표 크롤링

In [2]:
html = urlopen('http://en.wikipedia.org/wiki/Comparison_of_text_editors')
bs = BeautifulSoup(html, 'html.parser')

table = bs.findAll('table', {'class':'wikitable'})[0]
rows = table.findAll('tr')

In [3]:
#csvFile = open('editors.csv', 'wt+', encoding='UTF-8') 

# w: 쓰기 위해 열기, 파일을 먼저 자른다
# t: 텍스트 모드 (기본값)
# +:업데이트 (읽기 및 쓰기)를 위한 디스크 파일 열기

#writer = csv.writer(csvFile)

df = {}

#try :
colRow = []
for n, row in enumerate(rows) :    
    csvRow = []    
    for cell in row.findAll(['td', 'th']) : # td는 table data, th는 table header
        if n == 0 :
            colRow.append(cell.get_text().strip('\n'))
        else :
            csvRow.append(cell.get_text().strip('\n'))
        #print(csvRow)
    #writer.writerow(csvRow)
    if n == 0 :
        pass
    else :
        df[str(n)] = csvRow
#finally :
#    csvFile.close()

In [4]:
df_t = pd.DataFrame(df).T

In [5]:
df_t.columns = colRow

In [6]:
df_t.columns

Index(['', 'Creator', 'First public release', 'Latest stable version',
       'Latest Release Date', 'Programming language', 'Cost (US$)',
       'Software license', 'Open source', 'CLI available',
       'Minimum installed size'],
      dtype='object')

In [7]:
df_t.sort_values(by='First public release')

,,Creator,First public release,Latest stable version,Latest Release Date,Programming language,Cost (US$),Software license,Open source,CLI available,Minimum installed size
13,ed,Ken Thompson,1970,unchanged from original,,C,Free,?,,,0.04 MB
65,vi,Bill Joy,1976,3.7,,C,Free,BSD or CDDL,,,
64,VEDIT,"Ted Green, Greenview Data",1980,6.24.2,2015‑01‑12,"Assembly, C","$89 standard,239 Pro64",Proprietary,,,0.6 MB
49,Sam,Rob Pike,1980s (early),stable,,C,Free,LPL (OSI approved),,,
26,KEDIT,"Mansfield Software Group, Inc.",1983,1.6.1,2016‑12‑05,C,$129,Proprietary,,,1.1MB
...,...,...,...,...,...,...,...,...,...,...,...
32,Light Table,Chris Granger,2012,0.8.1,2016‑01‑21,ClojureScript,Free,MIT,,,
5,Atom,GitHub,2014,1.46.0,2020‑05‑05,"HTML, CSS, JavaScript, C++",Free,MIT,,,~ 107‑185 MB
12,CudaText,"UVViewSoft (alexey_t, kvichans, matthias030)",2015,1.57.0,2018‑06‑25,Object Pascal (Lazarus),Free,MPL 2.0,,,8 MB Mac\n12 MB Win\n5 Mb *nix
67,Visual Studio Code,Microsoft,2015,1.40.1,2019‑11‑13,"JavaScript, TypeScript",Free,MIT,,,~ 53‑82 MB


## 뉴스 크롤링

In [8]:
import requests

In [9]:
s_word = '모바일'
s_url = requests.get('https://search.naver.com/search.naver?query='+s_word+'&where=news&ie=utf8&sm=nws_hty', 
                     headers={'User-Agent':'Chrome/51.0.2704.103'})
bs = BeautifulSoup(s_url.text, 'html.parser')

# news_list = bs.select('ul.list_news > li') #news_list container 추출
news_links = bs.findAll('a', 'news_tit') # 뉴스 title 부분만 추출, 아예 직접 태그 지정 (link, title 정보 포함)

In [10]:
link_list = []
title_list = []
for i in news_links :
    link_list.append(i.attrs['href'])
    title_list.append(i.attrs['title'])
    
df = pd.DataFrame({'title':title_list, 'link':link_list})
df

,title,link
0,10일 전자문서법 시행…종이문서 없애고 모바일 전자고지 활성화,http://www.edaily.co.kr/news/newspath.asp?news...
1,"모바일 게임 '프로야구 H2', 감독 시스템 추가",http://www.fomos.kr/redirect/news_view?news_ca...
2,"엔픽셀, 모바일 신작 ‘그랑사가’ 사전등록 300만면 넘어",http://isplus.live.joins.com/news/article/arti...
3,"무안군-한국조폐공사, 모바일 무안사랑상품권 업무협약 체결",http://www.nbnnews.co.kr/news/articleView.html...
4,"방통위, 내년부터 UHD 다채널·모바일 시범 방송 허용",http://news.kbs.co.kr/news/view.do?ncd=5067325...
5,애큐온저축銀 ‘올해 최고의 모바일 앱’ 선정,http://daily.hankooki.com/lpage/finance/202012...
6,"신작 모바일 MMORPG '미르4', '10만 백룡 용사의 집결' 업데이트 실시",http://sports.chosun.com/news/ntype.htm?id=202...
7,모바일 소셜 카지노게임 다미포커 128개국 동시 출시,https://game.donga.com/97821/
8,롯데건설 '웹 어워드 코리아 2020' 모바일웹 최고대상,https://www.news1.kr/articles/?4142834
9,"펄어비스 검은사막모바일, 2021년 한정판 달력 출시",https://zdnet.co.kr/view/?no=20201208144348


https://hogni.tistory.com/55
* 1시간 마다 자동 전송 bot 만들기